# <center> Validation croisée K-Fold avec Ultralytics YOLOv8 </center> 

## Introduction

Ce guide complet illustre la mise en œuvre de la validation croisée K-Fold pour les ensembles de données de détection d'objets dans l'écosystème Ultralytics . Nous nous appuierons sur le format de détection YOLO et les principales bibliothèques Python telles que sklearn, pandas et PyYaml pour te guider dans la configuration nécessaire, le processus de génération de vecteurs de caractéristiques et l'exécution d'un fractionnement de l'ensemble de données K-Fold.

<img src="images/k-val.png" alt="Markdown Monster icon" height="80%" width="80%"/>

1. Commence par créer un nouveau fichier Python et importe les bibliothèques nécessaires

In [1]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

C:\Users\JulienNigou\AppData\Local\Temp\ipykernel_44332\1634132550.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


2. Procède à la récupération de tous les fichiers d'étiquettes de ton jeu de données.

In [2]:
dataset_path = Path("C:\\Users\\JulienNigou\\Desktop\\RockPaper\\Rock_Paper_Scissors_Data") # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt")) # all data in 'labels'

3. Maintenant, lis le contenu du fichier YAML du jeu de données et extrais les indices des étiquettes de classe.

In [3]:
yaml_file = dataset_path / "data.yaml"  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted(classes.keys())

4. Initialise un fichier vide pandas DataFrame

In [4]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

5. Compte les instances de chaque classe-étiquette présente dans les fichiers d'annotation.

In [5]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

C:\Users\JulienNigou\AppData\Local\Temp\ipykernel_44332\2598033761.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`


6. Voici un exemple de vue du DataFrame peuplé :

In [6]:
labels_df.head()

,0,1,2
Cool-Video-of-Ocean-Fish-Swimming_mp4-12_jpg.rf.c42aaf48a71c07bd20f047a10ef9add7,0.0,0.0,0.0
Cool-Video-of-Ocean-Fish-Swimming_mp4-8_jpg.rf.71ae8b64db1364bda1ace0dedae08284,0.0,0.0,0.0
egohands-public-1620849842064_png.rf.4c2b573a94e4360b9a5de54e5a5f7d5e,1.0,0.0,0.0
egohands-public-1620849844997_png.rf.75d751bd0fd9292d096bdc890924cacc,0.0,0.0,1.0
egohands-public-1620849847487_png.rf.cac65f49478a0af5a91fc30a2a602648,0.0,0.0,1.0


## Fractionnement de l'ensemble de données K-Fold

1. Nous allons maintenant utiliser le KFold de la classe sklearn.model_selection pour générer k les divisions de l'ensemble de données.
- Important :
    - Réglage shuffle=True assure une répartition aléatoire des classes dans tes fractionnements.
    - En réglant random_state=M où M est un nombre entier choisi, tu peux obtenir des résultats reproductibles

In [7]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

2. L'ensemble des données a maintenant été divisé en k plis, chacun ayant une liste de train et val indices. Nous allons construire un DataFrame pour afficher ces résultats plus clairement.

In [8]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

C:\Users\JulienNigou\AppData\Local\Temp\ipykernel_44332\3825890032.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
C:\Users\JulienNigou\AppData\Local\Temp\ipykernel_4433

3. Nous allons maintenant calculer la distribution des étiquettes de classe pour chaque pli sous la forme d'un ratio des classes présentes dans val aux personnes présentes dans train.

In [9]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

Le scénario idéal est que tous les ratios de classe soient raisonnablement similaires pour chaque fractionnement et pour toutes les classes. Cela dépend toutefois des spécificités de ton jeu de données.

4. Ensuite, nous créons les répertoires et les fichiers YAML des jeux de données pour chaque fractionnement.

In [10]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    for split in ['train', 'val']:#, 'test']:
        images.extend(sorted((dataset_path / split / 'images').rglob(f"*{ext}")))
    

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

5. Enfin, copie les images et les étiquettes dans le répertoire respectif ('train' ou 'val') pour chaque fractionnement.

- NOTE : Le temps nécessaire pour cette partie du code variera en fonction de la taille de ton ensemble de données et du matériel de ton système

In [11]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

## Sauvegarder les enregistrements (optionnel)

En option, tu peux enregistrer les enregistrements des DataFrames de fractionnement K-Fold et de distribution d'étiquettes sous forme de fichiers CSV pour référence ultérieure.

In [12]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

# Entraîne YOLO à l'aide de K-Fold Data Splits

In [13]:
import torch
import gc
from IPython.display import clear_output

1. Charge d'abord le modèle YOLO .

In [14]:
# # weights_path = 'path/to/weights.pt'
# # model = YOLO(weights_path, task='detect')
# model = YOLO('yolov8n.pt',task='detect').to(device)

2. Ensuite, itère sur les fichiers YAML de l'ensemble de données pour exécuter la formation. Les résultats seront enregistrés dans un répertoire spécifié par l'option project et name arguments. Par défaut, ce répertoire est 'exp/runs#' où # est un indice entier.

In [15]:
results = {}


# Define your additional arguments here
batch = 8
project = 'kfold_demo'
epochs = 2

for k in range(ksplit):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    dataset_yaml = ds_yamls[k]
    model = YOLO('yolov8n.pt',task='detect').to(device) # reset the weights
    model.train(data=dataset_yaml,epochs=epochs, batch=batch, project=project,cache=True)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis
    
    print("start cleanning")

    model = None
    gc.collect()
    del device
    clear_output(wait=True)
    torch.cuda.empty_cache()

engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\JulienNigou\Desktop\RockPaper\Rock_Paper_Scissors_Data\2024-02-08_5-Fold_Cross-val\split_2\split_2_dataset.yaml, epochs=2, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=cuda:0, workers=8, project=kfold_demo, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning C:\Users\JulienNigou\Desktop\RockPaper\Rock_Paper_Scissors_Data\2024-02-08_5-Fold_Cross-val\split_2\train\labels... 329 images, 246 backgrounds, 0 corrupt: 100%|██████████| 479/479 [00:00<00:00, 1482.45it/s]


train: New cache created: C:\Users\JulienNigou\Desktop\RockPaper\Rock_Paper_Scissors_Data\2024-02-08_5-Fold_Cross-val\split_2\train\labels.cache


train: Caching images (0.4GB True): 100%|██████████| 479/479 [00:00<00:00, 1380.74it/s]
val: Scanning C:\Users\JulienNigou\Desktop\RockPaper\Rock_Paper_Scissors_Data\2024-02-08_5-Fold_Cross-val\split_2\val\labels... 16 images, 102 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<00:00, 505.79it/s]

val: New cache created: C:\Users\JulienNigou\Desktop\RockPaper\Rock_Paper_Scissors_Data\2024-02-08_5-Fold_Cross-val\split_2\val\labels.cache



val: Caching images (0.1GB True): 100%|██████████| 114/114 [00:00<00:00, 491.36it/s]


# Conclusion

Dans ce guide, nous avons exploré le processus d'utilisation de la validation croisée K-Fold pour l'entraînement du modèle de détection d'objets YOLO . Nous avons appris à diviser notre ensemble de données en K partitions, en veillant à ce que la répartition des classes soit équilibrée entre les différents plis.

Nous avons également exploré la procédure de création de DataFrames de rapport pour visualiser les divisions de données et les distributions d'étiquettes sur ces divisions, ce qui nous a permis d'avoir un aperçu clair de la structure de nos ensembles de formation et de validation.

En option, nous avons sauvegardé nos enregistrements pour les consulter ultérieurement, ce qui pourrait être particulièrement utile dans les projets à grande échelle ou lors du dépannage des performances du modèle.

Enfin, nous avons mis en œuvre la formation réelle du modèle en utilisant chaque fractionnement dans une boucle, en sauvegardant nos résultats de formation pour une analyse et une comparaison ultérieures.

Cette technique de validation croisée K-Fold est un moyen robuste de tirer le meilleur parti de tes données disponibles, et elle permet de s'assurer que les performances de ton modèle sont fiables et cohérentes dans les différents sous-ensembles de données. Il en résulte un modèle plus généralisable et plus fiable, moins susceptible de s'adapter de façon excessive à des modèles de données spécifiques.

N'oublie pas que même si nous avons utilisé YOLO dans ce guide, ces étapes sont en grande partie transférables à d'autres modèles d'apprentissage automatique. Comprendre ces étapes te permet d'appliquer efficacement la validation croisée dans tes propres projets d'apprentissage automatique. Bon codage !